In [66]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from ipywidgets import interactive, Layout, Output
from IPython.display import display, clear_output
from datetime import datetime

# Enable inline Matplotlib plots
%matplotlib inline

# Create date pickers for the date range filter
start_date_picker = widgets.DatePicker(
    description='Start Date:',
    value=pd.Timestamp('2021-01-01'),
    disabled=False
)
end_date_picker = widgets.DatePicker(
    description='End Date:',
    value=pd.Timestamp('2022-04-24'),
    disabled=False
)
# Add a checkbox widget for the "Total" option
total_checkbox = widgets.Checkbox(
    description='Merge Platforms',
    value=False,  # By default, the checkbox is unchecked
    disabled=False
)   

# Create a function to display a plot
def display_download_platform():
    # Load the dataset
    df = pd.read_csv('D:\\MetroCar\\CSV\\funneldatawithdailydates.csv')
    df = df[df['funnel_step'] == 1]
    # Convert the 'daily_date' column to datetime
    df['daily_date'] = pd.to_datetime(df['daily_date'], format='%Y-%m-%d')

    # Filter the dataset based on the date range
    def filter_data(start_date, end_date, start_platform, end_platform):
        start_date = pd.Timestamp(start_date).date()
        end_date = pd.Timestamp(end_date).date()
        filtered_df = df[
        (df['daily_date'].dt.date >= start_date) & 
        (df['daily_date'].dt.date <= end_date)  & 
        (df['platform'] >= start_platform) &
        (df['platform'] <= end_platform)]
        return filtered_df

    # Define your default start and end platform ranges
    default_start_platform_range = 'android'
    default_end_platform_range = 'web'
    
    start_platform_range_picker = widgets.Dropdown(
        description='Start Platform:',
        options=df['platform'].unique(),
        value=default_start_platform_range,
        disabled=False
    )
    end_platform_range_picker = widgets.Dropdown(
        description='End Platform:',
        options=df['platform'].unique(),
        value=default_end_platform_range,
        disabled=False
    )    
    
# Update function for the line plot
    def update_line_plot(start_date, end_date, start_platform, end_platform, total):
        if total:
        # If the "Total" checkbox is checked, calculate the total users by date
            filtered_data = filter_data(start_date, end_date, start_platform, end_platform)
            total_data = filtered_data.groupby('daily_date')['users'].sum().reset_index()

            fig = go.Figure()
            fig.add_trace(go.Scatter(x=total_data['daily_date'], y=total_data['users'], name='Total'))
        else:
        # If "Total" is unchecked, display the data per platform
            filtered_data = filter_data(start_date, end_date, start_platform, end_platform)

            fig = go.Figure()

            platforms = filtered_data['platform'].unique()

            for platform in platforms:
                platform_data = filtered_data[filtered_data['platform'] == platform]
                platform_data_grouped = platform_data.groupby('daily_date')['users'].sum().reset_index()

                fig.add_trace(go.Scatter(x=platform_data_grouped['daily_date'], y=platform_data_grouped['users'], name=platform))

        fig.update_layout(title="Number of Users per Platform",
                              xaxis_title="Date",
                              yaxis_title="Number of Users")

        fig.show()

    # Create an interactive widget to update the line plot
    widget = interactive(
        update_line_plot, 
        start_date=start_date_picker, 
        end_date=end_date_picker,
        start_platform=start_platform_range_picker,
        end_platform=end_platform_range_picker,
        total=total_checkbox
    )

    # Display the widgets
    display(widget)




In [67]:
def display_signups_visualization():
    # Load the dataset
    df = pd.read_csv('D:\\MetroCar\\CSV\\funneldatawithdailydates.csv')
    df = df[df['funnel_step'] == 2]
    
    # Convert the 'daily_date' column to datetime
    df['daily_date'] = pd.to_datetime(df['daily_date'], format='%Y-%m-%d')

    # Filter the dataset based on the date range and age range
    def filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform):
        start_date = pd.Timestamp(start_date).date()
        end_date = pd.Timestamp(end_date).date()
        filtered_df = df[
            (df['daily_date'].dt.date >= start_date) &
            (df['daily_date'].dt.date <= end_date) &
            (df['age_range'] >= start_age_range) &
            (df['age_range'] <= end_age_range) &
            (df['platform'] >= start_platform) &
            (df['platform'] <= end_platform)
        ]
        return filtered_df
    # Filter out 'Unknown' values from the age_range column
    filtered_age_ranges = df[df['age_range'] != 'Unknown']['age_range']

    # Sort the unique age range values with a custom sorting function
    unique_age_ranges = sorted(filtered_age_ranges.unique(), key=lambda x: [int(age) for age in x.split('-')])
    # Append 'Unknown' to the end of the sorted list
    unique_age_ranges.append('Unknown')     

    # Define your default start and end age ranges
    default_start_age_range = '18-24'
    default_end_age_range = 'Unknown'
    
    start_age_range_picker = widgets.Dropdown(
        description='Start Age Range:',
        options=unique_age_ranges,
        value=default_start_age_range,
        disabled=False
    )
    end_age_range_picker = widgets.Dropdown(
        description='End Age Range:',
        options=unique_age_ranges,
        value=default_end_age_range,
        disabled=False
    )
    # Define your default start and end platform ranges
    default_start_platform_range = 'android'
    default_end_platform_range = 'web'
    
    start_platform_range_picker = widgets.Dropdown(
        description='Start Platform:',
        options=df['platform'].unique(),
        value=default_start_platform_range,
        disabled=False
    )
    end_platform_range_picker = widgets.Dropdown(
        description='End Platform:',
        options=df['platform'].unique(),
        value=default_end_platform_range,
        disabled=False
    )

    # Update function for the bar chart
    def update_bar_chart(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform, total):
        if total:
            # If the "Total" checkbox is checked, calculate the total users
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)
            total_data = filtered_data.groupby(['platform'])['users'].sum().reset_index()

            fig = go.Figure()
            fig.add_trace(go.Bar(x=total_data['platform'], y=total_data['users'], name='Total'))
        else:
            # If "Total" is unchecked, display the data per platform
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)

            fig = go.Figure()

            platforms = filtered_data['platform'].unique()
            age_ranges = filtered_data['age_range'].unique()

            for age_range in age_ranges:
                age_range_data = filtered_data[filtered_data['age_range'] == age_range]
                users_by_platform = []

                for platform in platforms:
                    platform_data = age_range_data[age_range_data['platform'] == platform]
                    users = platform_data['users'].sum()
                    users_by_platform.append(users)

                # Create a label that includes both age_range and platform information
                label = [f'{age_range} - {platform}' for platform in platforms]

                fig.add_trace(go.Bar(x=label, y=users_by_platform, name=age_range))

        fig.update_layout(title="Number of Users per Platform and Age Range",
                          xaxis_title="Age Range - Platform",
                          yaxis_title="Number of Users")

        fig.show()
    # Create an interactive widget to update the bar chart
    widget = interactive(
        update_bar_chart, 
        start_date=start_date_picker, 
        end_date=end_date_picker, 
        start_age_range=start_age_range_picker, 
        end_age_range=end_age_range_picker,
        start_platform=start_platform_range_picker,
        end_platform=end_platform_range_picker,
        total=total_checkbox
    )

    # Display the widgets
    display(widget)



In [68]:
# Create a function to display a plot
def display_signupsdaily_visualization():
    # Load the dataset
    df = pd.read_csv('D:\\MetroCar\\CSV\\funneldatawithdailydates.csv')
    df = df[df['funnel_step'] == 2]
    # Convert the 'daily_date' column to datetime
    df['daily_date'] = pd.to_datetime(df['daily_date'], format='%Y-%m-%d')

    # Filter the dataset based on the date range
    def filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform):
        start_date = pd.Timestamp(start_date).date()
        end_date = pd.Timestamp(end_date).date()
        filtered_df = df[
        (df['daily_date'].dt.date >= start_date) & 
        (df['daily_date'].dt.date <= end_date) &
        (df['age_range'] >= start_age_range) & 
        (df['age_range'] <= end_age_range) &        
        (df['platform'] >= start_platform) &
        (df['platform'] <= end_platform)
        ]
        return filtered_df

    # Filter out 'Unknown' values from the age_range column
    filtered_age_ranges = df[df['age_range'] != 'Unknown']['age_range']

    # Sort the unique age range values with a custom sorting function
    unique_age_ranges = sorted(filtered_age_ranges.unique(), key=lambda x: [int(age) for age in x.split('-')])
    # Append 'Unknown' to the end of the sorted list
    unique_age_ranges.append('Unknown')     

    # Define your default start and end age ranges
    default_start_age_range = '18-24'
    default_end_age_range = 'Unknown'
    
    start_age_range_picker = widgets.Dropdown(
        description='Start Age Range:',
        options=unique_age_ranges,
        value=default_start_age_range,
        disabled=False
    )
    end_age_range_picker = widgets.Dropdown(
        description='End Age Range:',
        options=unique_age_ranges,
        value=default_end_age_range,
        disabled=False
    )    
    # Define your default start and end platform ranges
    default_start_platform_range = 'android'
    default_end_platform_range = 'web'
        
    start_platform_range_picker = widgets.Dropdown(
        description='Start Platform:',
        options=df['platform'].unique(),
        value=default_start_platform_range,
        disabled=False
    )
    end_platform_range_picker = widgets.Dropdown(
        description='End Platform:',
        options=df['platform'].unique(),
        value=default_end_platform_range,
        disabled=False
    )

# Update function for the line plot
    def update_line_plot(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform, total):
        if total:
        # If the "Total" checkbox is checked, calculate the total users by date
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)
            total_data = filtered_data.groupby('daily_date')['users'].sum().reset_index()

            fig = go.Figure()
            fig.add_trace(go.Scatter(x=total_data['daily_date'], y=total_data['users'], name='Total'))
        else:
        # If "Total" is unchecked, display the data per platform
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)

            fig = go.Figure()

            platforms = filtered_data['platform'].unique()

            for platform in platforms:
                platform_data = filtered_data[filtered_data['platform'] == platform]
                platform_data_grouped = platform_data.groupby('daily_date')['users'].sum().reset_index()

                fig.add_trace(go.Scatter(x=platform_data_grouped['daily_date'], y=platform_data_grouped['users'], name=platform))

        fig.update_layout(title="Number of Users per Platform",
                              xaxis_title="Date",
                              yaxis_title="Number of Users")

        fig.show()

    # Create an interactive widget to update the line plot
    widget = interactive(
        update_line_plot, 
        start_date=start_date_picker, 
        end_date=end_date_picker, 
        start_age_range=start_age_range_picker, 
        end_age_range=end_age_range_picker,        
        start_platform=start_platform_range_picker, 
        end_platform=end_platform_range_picker,
        total=total_checkbox
    )

    # Display the widgets
    display(widget)
    


In [69]:
# Create a function to display a Funnel Chart
def display_funnel_visualization():
    # Read data from CSV file
    df = pd.read_csv('D:\\MetroCar\\CSV\\funnelnoage.csv')
    # Sort the DataFrame by funnel step in descending order to have funnel 1 on top
    df = df.sort_values(by="funnel_step", ascending=False)

    # Pivot the data for plotting, including funnel_names
    pivot_df = df.pivot(index="funnel_step", columns="platform", values="users")
    pivot_df_names = df.pivot(index="funnel_step", columns="platform", values="funnel_name")

    # Specify colors for each platform
    colors = {
        "android": "rgb(0, 123, 255)",
        "ios": "rgb(40, 167, 69)",
        "web": "rgb(220, 53, 69)",
    }
    # Calculate the width and height in pixels based on the desired figsize in inches and DPI
    figsize_inches = (8.27, 5.87)
    dpi = 96
    width_pixels = int(figsize_inches[0] * dpi)
    height_pixels = int(figsize_inches[1] * dpi)

    # Create a Plotly funnel chart with specified colors
    fig = go.Figure()

    # Group the data by platform
    for platform in pivot_df.columns:
        fig.add_trace(go.Funnel(
            name=platform,
            y=pivot_df_names[platform],
            x=pivot_df[platform],
            textinfo="value+percent initial",
            textposition="inside",
            opacity=0.75,
            marker=dict(color=colors[platform]),
        ))

    # Set the layout
    fig.update_layout(
        title="MetroCar Funnel By Platforms",
        #yaxis_title="Funnel Name",
        legend_title="Platform",
        width=width_pixels,
        height=height_pixels,
    )

    # Show the interactive Plotly chart
    fig.show()
    # Transform the interactive Plotly chart into an Html and save it.
    #fig.write_html("interactive_funnel_chart.html")


In [70]:
def display_riderequests_visualization():
    # Load the dataset
    df = pd.read_csv('D:\\MetroCar\\CSV\\funneldatawithdailydates.csv')
    df = df[df['funnel_step'] == 3]
    
    # Convert the 'daily_date' column to datetime
    df['daily_date'] = pd.to_datetime(df['daily_date'], format='%Y-%m-%d')

    # Filter the dataset based on the date range and age range
    def filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform):
        start_date = pd.Timestamp(start_date).date()
        end_date = pd.Timestamp(end_date).date()
        filtered_df = df[
            (df['daily_date'].dt.date >= start_date) &
            (df['daily_date'].dt.date <= end_date) &
            (df['age_range'] >= start_age_range) &
            (df['age_range'] <= end_age_range) &
            (df['platform'] >= start_platform) &
            (df['platform'] <= end_platform)
        ]
        return filtered_df
    # Filter out 'Unknown' values from the age_range column
    filtered_age_ranges = df[df['age_range'] != 'Unknown']['age_range']

    # Sort the unique age range values with a custom sorting function
    unique_age_ranges = sorted(filtered_age_ranges.unique(), key=lambda x: [int(age) for age in x.split('-')])
    # Append 'Unknown' to the end of the sorted list
    unique_age_ranges.append('Unknown')     

    # Define your default start and end age ranges
    default_start_age_range = '18-24'
    default_end_age_range = 'Unknown'
    
    start_age_range_picker = widgets.Dropdown(
        description='Start Age Range:',
        options=unique_age_ranges,
        value=default_start_age_range,
        disabled=False
    )
    end_age_range_picker = widgets.Dropdown(
        description='End Age Range:',
        options=unique_age_ranges,
        value=default_end_age_range,
        disabled=False
    )
    # Define your default start and end platform ranges
    default_start_platform_range = 'android'
    default_end_platform_range = 'web'
    
    start_platform_range_picker = widgets.Dropdown(
        description='Start Platform:',
        options=df['platform'].unique(),
        value=default_start_platform_range,
        disabled=False
    )
    end_platform_range_picker = widgets.Dropdown(
        description='End Platform:',
        options=df['platform'].unique(),
        value=default_end_platform_range,
        disabled=False
    )
    # Update function for the bar chart
    def update_bar_chart(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform, total):
        if total:
            # If the "Total" checkbox is checked, calculate the total users
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)
            total_data = filtered_data.groupby(['platform'])['rides'].sum().reset_index()

            fig = go.Figure()
            fig.add_trace(go.Bar(x=total_data['platform'], y=total_data['rides'], name='Total'))
        else:
            # If "Total" is unchecked, display the data per platform
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)

            fig = go.Figure()

            platforms = filtered_data['platform'].unique()
            age_ranges = filtered_data['age_range'].unique()

            for age_range in age_ranges:
                age_range_data = filtered_data[filtered_data['age_range'] == age_range]
                rides_by_platform = []

                for platform in platforms:
                    platform_data = age_range_data[age_range_data['platform'] == platform]
                    rides = platform_data['rides'].sum()
                    rides_by_platform.append(rides)

                # Create a label that includes both age_range and platform information
                label = [f'{age_range} - {platform}' for platform in platforms]

                fig.add_trace(go.Bar(x=label, y=rides_by_platform, name=age_range))

        fig.update_layout(title="Number of Rides per Platform and Age Range",
                          xaxis_title="Age Range - Platform",
                          yaxis_title="Number of Rides")

        fig.show()
    # Create an interactive widget to update the bar chart
    widget = interactive(
        update_bar_chart, 
        start_date=start_date_picker, 
        end_date=end_date_picker, 
        start_age_range=start_age_range_picker, 
        end_age_range=end_age_range_picker,
        start_platform=start_platform_range_picker,
        end_platform=end_platform_range_picker,
        total=total_checkbox
    )

    # Display the widgets
    display(widget)



In [71]:
# Create a function to display a plot
def display_rides_cancellations_visualization():
    # Load the dataset
    df = pd.read_csv('D:\\MetroCar\\CSV\\funneldatawithdailydates.csv')
    df = df[df['funnel_step'] == 3]
    # Convert the 'daily_date' column to datetime
    df['daily_date'] = pd.to_datetime(df['daily_date'], format='%Y-%m-%d')

    # Filter the dataset based on the date range
    def filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform):
        start_date = pd.Timestamp(start_date).date()
        end_date = pd.Timestamp(end_date).date()
        filtered_df = df[
        (df['daily_date'].dt.date >= start_date) & 
        (df['daily_date'].dt.date <= end_date) &
        (df['age_range'] >= start_age_range) & 
        (df['age_range'] <= end_age_range) &        
        (df['platform'] >= start_platform) &
        (df['platform'] <= end_platform)
        ]
        return filtered_df
    # Filter out 'Unknown' values from the age_range column
    filtered_age_ranges = df[df['age_range'] != 'Unknown']['age_range']

    # Sort the unique age range values with a custom sorting function
    unique_age_ranges = sorted(filtered_age_ranges.unique(), key=lambda x: [int(age) for age in x.split('-')])
    # Append 'Unknown' to the end of the sorted list
    unique_age_ranges.append('Unknown')     

    # Define your default start and end age ranges
    default_start_age_range = '18-24'
    default_end_age_range = 'Unknown'
    
    start_age_range_picker = widgets.Dropdown(
        description='Start Age Range:',
        options=unique_age_ranges,
        value=default_start_age_range,
        disabled=False
    )
    end_age_range_picker = widgets.Dropdown(
        description='End Age Range:',
        options=unique_age_ranges,
        value=default_end_age_range,
        disabled=False
    )    
    # Define your default start and end platform ranges
    default_start_platform_range = 'android'
    default_end_platform_range = 'web'
        
    start_platform_range_picker = widgets.Dropdown(
        description='Start Platform:',
        options=df['platform'].unique(),
        value=default_start_platform_range,
        disabled=False
    )
    end_platform_range_picker = widgets.Dropdown(
        description='End Platform:',
        options=df['platform'].unique(),
        value=default_end_platform_range,
        disabled=False
    )

    # Update function for the line plot
    def update_line_plot(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform, total):
        if total:
            # If the "Merge Platforms" checkbox is checked, calculate the total rides and cancellations
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)
            total_rides = filtered_data.groupby('daily_date')['rides'].sum().reset_index()
            total_cancellations = filtered_data.groupby('daily_date')['cancellations'].sum().reset_index()
        
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=total_rides['daily_date'], y=total_rides['rides'], name='Rides'))
            fig.add_trace(go.Scatter(x=total_cancellations['daily_date'], y=total_cancellations['cancellations'], name='Cancellations'))
        else:
            # If "Merge Platforms" is unchecked, display the data per platform
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)

            fig = go.Figure()

            platforms = filtered_data['platform'].unique()
        
            for platform in platforms:
                platform_data = filtered_data[filtered_data['platform'] == platform]
                platform_data_grouped = platform_data.groupby('daily_date')[['rides', 'cancellations']].sum().reset_index()
                fig.add_trace(go.Scatter(x=platform_data_grouped['daily_date'], y=platform_data_grouped['rides'], name=f'{platform} Rides'))
                fig.add_trace(go.Scatter(x=platform_data_grouped['daily_date'], y=platform_data_grouped['cancellations'], name=f'{platform} Cancellations'))
    
        fig.update_layout(title="Number of Rides and Cancellations per Platform",
                          xaxis_title="Date",
                          yaxis_title="Counts")
    
        fig.show()

    # Create an interactive widget to update the line plot
    widget = interactive(
        update_line_plot, 
        start_date=start_date_picker, 
        end_date=end_date_picker, 
        start_age_range=start_age_range_picker, 
        end_age_range=end_age_range_picker,        
        start_platform=start_platform_range_picker, 
        end_platform=end_platform_range_picker,
        total=total_checkbox
    )

    # Display the widgets
    display(widget)
    


In [72]:
def display_completed_age_platform():
    # Load the dataset
    df = pd.read_csv('D:\\MetroCar\\CSV\\funneldatawithdailydates.csv')
    df = df[df['funnel_step'] == 4]
    
    # Convert the 'daily_date' column to datetime
    df['daily_date'] = pd.to_datetime(df['daily_date'], format='%Y-%m-%d')

    # Filter the dataset based on the date range and age range
    def filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform):
        start_date = pd.Timestamp(start_date).date()
        end_date = pd.Timestamp(end_date).date()
        filtered_df = df[
            (df['daily_date'].dt.date >= start_date) &
            (df['daily_date'].dt.date <= end_date) &
            (df['age_range'] >= start_age_range) &
            (df['age_range'] <= end_age_range) &
            (df['platform'] >= start_platform) &
            (df['platform'] <= end_platform)
        ]
        return filtered_df

    # Filter out 'Unknown' values from the age_range column
    filtered_age_ranges = df[df['age_range'] != 'Unknown']['age_range']

    # Sort the unique age range values with a custom sorting function
    unique_age_ranges = sorted(filtered_age_ranges.unique(), key=lambda x: [int(age) for age in x.split('-')])
    # Append 'Unknown' to the end of the sorted list
    unique_age_ranges.append('Unknown')     

    # Define your default start and end age ranges
    default_start_age_range = '18-24'
    default_end_age_range = 'Unknown'
    
    start_age_range_picker = widgets.Dropdown(
        description='Start Age Range:',
        options=unique_age_ranges,
        value=default_start_age_range,
        disabled=False
    )
    end_age_range_picker = widgets.Dropdown(
        description='End Age Range:',
        options=unique_age_ranges,
        value=default_end_age_range,
        disabled=False
    )
    # Define your default start and end platform ranges
    default_start_platform_range = 'android'
    default_end_platform_range = 'web'
    
    start_platform_range_picker = widgets.Dropdown(
        description='Start Platform:',
        options=df['platform'].unique(),
        value=default_start_platform_range,
        disabled=False
    )
    end_platform_range_picker = widgets.Dropdown(
        description='End Platform:',
        options=df['platform'].unique(),
        value=default_end_platform_range,
        disabled=False
    )
    # Update function for the bar chart
    def update_bar_chart(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform, total):
        if total:
            # If the "Total" checkbox is checked, calculate the total users
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)
            total_data = filtered_data.groupby(['platform'])['rides'].sum().reset_index()

            fig = go.Figure()
            fig.add_trace(go.Bar(x=total_data['platform'], y=total_data['rides'], name='Total'))
        else:
            # If "Total" is unchecked, display the data per platform
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)

            fig = go.Figure()

            platforms = filtered_data['platform'].unique()
            age_ranges = filtered_data['age_range'].unique()

            for age_range in age_ranges:
                age_range_data = filtered_data[filtered_data['age_range'] == age_range]
                rides_by_platform = []

                for platform in platforms:
                    platform_data = age_range_data[age_range_data['platform'] == platform]
                    rides = platform_data['rides'].sum()
                    rides_by_platform.append(rides)

                # Create a label that includes both age_range and platform information
                label = [f'{age_range} - {platform}' for platform in platforms]

                fig.add_trace(go.Bar(x=label, y=rides_by_platform, name=age_range))

        fig.update_layout(title="Number of Rides per Platform and Age Range",
                          xaxis_title="Age Range - Platform",
                          yaxis_title="Number of Rides")

        fig.show()
    # Create an interactive widget to update the bar chart
    widget = interactive(
        update_bar_chart, 
        start_date=start_date_picker, 
        end_date=end_date_picker, 
        start_age_range=start_age_range_picker, 
        end_age_range=end_age_range_picker,
        start_platform=start_platform_range_picker,
        end_platform=end_platform_range_picker,
        total=total_checkbox
    )

    # Display the widgets
    display(widget)



In [73]:
def display_completed_usd():
    # Load the dataset
    df = pd.read_csv('D:\\MetroCar\\CSV\\funneldatawithdailydates.csv')
    df = df[df['funnel_step'] == 4]
    
    # Convert the 'daily_date' column to datetime
    df['daily_date'] = pd.to_datetime(df['daily_date'], format='%Y-%m-%d')

    # Filter the dataset based on the date range and age range
    def filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform):
        start_date = pd.Timestamp(start_date).date()
        end_date = pd.Timestamp(end_date).date()
        filtered_df = df[
            (df['daily_date'].dt.date >= start_date) &
            (df['daily_date'].dt.date <= end_date) &
            (df['age_range'] >= start_age_range) &
            (df['age_range'] <= end_age_range) &
            (df['platform'] >= start_platform) &
            (df['platform'] <= end_platform)
        ]
        return filtered_df
  
    # Filter out 'Unknown' values from the age_range column
    filtered_age_ranges = df[df['age_range'] != 'Unknown']['age_range']

    # Sort the unique age range values with a custom sorting function
    unique_age_ranges = sorted(filtered_age_ranges.unique(), key=lambda x: [int(age) for age in x.split('-')])
    # Append 'Unknown' to the end of the sorted list
    unique_age_ranges.append('Unknown')     

    # Define your default start and end age ranges
    default_start_age_range = '18-24'
    default_end_age_range = 'Unknown'
    
    start_age_range_picker = widgets.Dropdown(
        description='Start Age Range:',
        options=unique_age_ranges,
        value=default_start_age_range,
        disabled=False
    )
    end_age_range_picker = widgets.Dropdown(
        description='End Age Range:',
        options=unique_age_ranges,
        value=default_end_age_range,
        disabled=False
    )
    # Define your default start and end platform ranges
    default_start_platform_range = 'android'
    default_end_platform_range = 'web'
    
    start_platform_range_picker = widgets.Dropdown(
        description='Start Platform:',
        options=df['platform'].unique(),
        value=default_start_platform_range,
        disabled=False
    )
    end_platform_range_picker = widgets.Dropdown(
        description='End Platform:',
        options=df['platform'].unique(),
        value=default_end_platform_range,
        disabled=False
    )

    # Update function for the bar chart
    def update_bar_chart(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform, total):
        if total:
            # If the "Total" checkbox is checked, calculate the total users
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)
            total_data = filtered_data.groupby(['platform'])['total_usd'].sum().reset_index()

            fig = go.Figure()
            fig.add_trace(go.Bar(x=total_data['platform'], y=total_data['total_usd'], name='USD'))
        else:
            # If "Total" is unchecked, display the data per platform
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)

            fig = go.Figure()

            platforms = filtered_data['platform'].unique()
            age_ranges = filtered_data['age_range'].unique()

            for age_range in age_ranges:
                age_range_data = filtered_data[filtered_data['age_range'] == age_range]
                usd_by_platform = []

                for platform in platforms:
                    platform_data = age_range_data[age_range_data['platform'] == platform]
                    usd = platform_data['total_usd'].sum()
                    usd_by_platform.append(usd)

                # Create a label that includes both age_range and platform information
                label = [f'{age_range} - {platform}' for platform in platforms]

                fig.add_trace(go.Bar(x=label, y=usd_by_platform, name=age_range))

        fig.update_layout(title="USD by Platform & Age Range",
                          xaxis_title="Age Range - Platform",
                          yaxis_title="USD")

        fig.show()
    # Create an interactive widget to update the bar chart
    widget = interactive(
        update_bar_chart, 
        start_date=start_date_picker, 
        end_date=end_date_picker, 
        start_age_range=start_age_range_picker, 
        end_age_range=end_age_range_picker,
        start_platform=start_platform_range_picker,
        end_platform=end_platform_range_picker,
        total=total_checkbox
    )

    # Display the widgets
    display(widget)



In [74]:
# Create a function to display a plot
def display_completed_usd_daily():
    # Load the dataset
    df = pd.read_csv('D:\\MetroCar\\CSV\\funneldatawithdailydates.csv')
    df = df[df['funnel_step'] == 4]
    # Convert the 'daily_date' column to datetime
    df['daily_date'] = pd.to_datetime(df['daily_date'], format='%Y-%m-%d')

    # Filter the dataset based on the date range
    def filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform):
        start_date = pd.Timestamp(start_date).date()
        end_date = pd.Timestamp(end_date).date()
        filtered_df = df[
        (df['daily_date'].dt.date >= start_date) & 
        (df['daily_date'].dt.date <= end_date) &
        (df['age_range'] >= start_age_range) & 
        (df['age_range'] <= end_age_range) &        
        (df['platform'] >= start_platform) &
        (df['platform'] <= end_platform)
        ]
        return filtered_df

    # Filter out 'Unknown' values from the age_range column
    filtered_age_ranges = df[df['age_range'] != 'Unknown']['age_range']

    # Sort the unique age range values with a custom sorting function
    unique_age_ranges = sorted(filtered_age_ranges.unique(), key=lambda x: [int(age) for age in x.split('-')])
    # Append 'Unknown' to the end of the sorted list
    unique_age_ranges.append('Unknown')     

    # Define your default start and end age ranges
    default_start_age_range = '18-24'
    default_end_age_range = 'Unknown'
    
    start_age_range_picker = widgets.Dropdown(
        description='Start Age Range:',
        options=unique_age_ranges,
        value=default_start_age_range,
        disabled=False
    )
    end_age_range_picker = widgets.Dropdown(
        description='End Age Range:',
        options=unique_age_ranges,
        value=default_end_age_range,
        disabled=False
    )    
    # Define your default start and end platform ranges
    default_start_platform_range = 'android'
    default_end_platform_range = 'web'
        
    start_platform_range_picker = widgets.Dropdown(
        description='Start Platform:',
        options=df['platform'].unique(),
        value=default_start_platform_range,
        disabled=False
    )
    end_platform_range_picker = widgets.Dropdown(
        description='End Platform:',
        options=df['platform'].unique(),
        value=default_end_platform_range,
        disabled=False
    )

    # Update function for the line plot
    def update_line_plot(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform, total):
        if total:
            # If the "Merge Platforms" checkbox is checked, calculate the total rides and cancellations
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)
            total_rides = filtered_data.groupby('daily_date')['rides'].sum().reset_index()
            total_usd = filtered_data.groupby('daily_date')['total_usd'].sum().reset_index()
        
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=total_rides['daily_date'], y=total_rides['rides'], name='Rides'))
            fig.add_trace(go.Scatter(x=total_usd['daily_date'], y=total_usd['total_usd'], name='USD'))
        else:
            # If "Merge Platforms" is unchecked, display the data per platform
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)

            fig = go.Figure()

            platforms = filtered_data['platform'].unique()
        
            for platform in platforms:
                platform_data = filtered_data[filtered_data['platform'] == platform]
                platform_data_grouped = platform_data.groupby('daily_date')[['rides', 'total_usd']].sum().reset_index()
                fig.add_trace(go.Scatter(x=platform_data_grouped['daily_date'], y=platform_data_grouped['rides'], name=f'{platform} Rides'))
                fig.add_trace(go.Scatter(x=platform_data_grouped['daily_date'], y=platform_data_grouped['total_usd'], name=f'{platform} USD'))
    
        fig.update_layout(title="Number of Rides and USD per Platform",
                          xaxis_title="Date",
                          yaxis_title="Counts")
    
        fig.show()

    # Create an interactive widget to update the line plot
    widget = interactive(
        update_line_plot, 
        start_date=start_date_picker, 
        end_date=end_date_picker, 
        start_age_range=start_age_range_picker, 
        end_age_range=end_age_range_picker,        
        start_platform=start_platform_range_picker, 
        end_platform=end_platform_range_picker,
        total=total_checkbox
    )

    # Display the widgets
    display(widget)
    


In [75]:
def display_completed_ratings():
    # Load the dataset
    df = pd.read_csv('D:\\MetroCar\\CSV\\funneldatawithdailydates.csv')
    df = df[df['funnel_step'] == 4]
    
    # Convert the 'daily_date' column to datetime
    df['daily_date'] = pd.to_datetime(df['daily_date'], format='%Y-%m-%d')

    # Filter the dataset based on the date range and age range
    def filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform):
        start_date = pd.Timestamp(start_date).date()
        end_date = pd.Timestamp(end_date).date()
        filtered_df = df[
            (df['daily_date'].dt.date >= start_date) &
            (df['daily_date'].dt.date <= end_date) &
            (df['age_range'] >= start_age_range) &
            (df['age_range'] <= end_age_range) &
            (df['platform'] >= start_platform) &
            (df['platform'] <= end_platform)
        ]
        return filtered_df
  
    # Filter out 'Unknown' values from the age_range column
    filtered_age_ranges = df[df['age_range'] != 'Unknown']['age_range']

    # Sort the unique age range values with a custom sorting function
    unique_age_ranges = sorted(filtered_age_ranges.unique(), key=lambda x: [int(age) for age in x.split('-')])
    # Append 'Unknown' to the end of the sorted list
    unique_age_ranges.append('Unknown')     

    # Define your default start and end age ranges
    default_start_age_range = '18-24'
    default_end_age_range = 'Unknown'
    
    start_age_range_picker = widgets.Dropdown(
        description='Start Age Range:',
        options=unique_age_ranges,
        value=default_start_age_range,
        disabled=False
    )
    end_age_range_picker = widgets.Dropdown(
        description='End Age Range:',
        options=unique_age_ranges,
        value=default_end_age_range,
        disabled=False
    )
    # Define your default start and end platform ranges
    default_start_platform_range = 'android'
    default_end_platform_range = 'web'
    
    start_platform_range_picker = widgets.Dropdown(
        description='Start Platform:',
        options=df['platform'].unique(),
        value=default_start_platform_range,
        disabled=False
    )
    end_platform_range_picker = widgets.Dropdown(
        description='End Platform:',
        options=df['platform'].unique(),
        value=default_end_platform_range,
        disabled=False
    )

    # Update function for the bar chart
    def update_bar_chart(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform, total):
        if total:
            # If the "Total" checkbox is checked, calculate the total users
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)
            mean_data = filtered_data.groupby(['platform'])['avg_rating'].mean().reset_index()

            fig = go.Figure()
            fig.add_trace(go.Bar(x=mean_data['platform'], y=mean_data['avg_rating'], name='USD'))
        else:
            # If "Total" is unchecked, display the data per platform
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)

            fig = go.Figure()

            platforms = filtered_data['platform'].unique()
            age_ranges = filtered_data['age_range'].unique()

            for age_range in age_ranges:
                age_range_data = filtered_data[filtered_data['age_range'] == age_range]
                ratings_by_platform = []

                for platform in platforms:
                    platform_data = age_range_data[age_range_data['platform'] == platform]
                    ratings = platform_data['avg_rating'].mean()
                    ratings_by_platform.append(ratings)

                # Create a label that includes both age_range and platform information
                label = [f'{age_range} - {platform}' for platform in platforms]

                fig.add_trace(go.Bar(x=label, y=ratings_by_platform, name=age_range))

        fig.update_layout(title="Average Rating by Platform & Age Range",
                          xaxis_title="Age Range - Platform",
                          yaxis_title="Average Rating")

        fig.show()
    # Create an interactive widget to update the bar chart
    widget = interactive(
        update_bar_chart, 
        start_date=start_date_picker, 
        end_date=end_date_picker, 
        start_age_range=start_age_range_picker, 
        end_age_range=end_age_range_picker,
        start_platform=start_platform_range_picker,
        end_platform=end_platform_range_picker,
        total=total_checkbox
    )

    # Display the widgets
    display(widget)



In [76]:
# Create a function to display a plot
def display_completed_reviews_daily():
    # Load the dataset
    df = pd.read_csv('D:\\MetroCar\\CSV\\funneldatawithdailydates.csv')
    df = df[df['funnel_step'] == 4]
    # Convert the 'daily_date' column to datetime
    df['daily_date'] = pd.to_datetime(df['daily_date'], format='%Y-%m-%d')

    # Filter the dataset based on the date range
    def filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform):
        start_date = pd.Timestamp(start_date).date()
        end_date = pd.Timestamp(end_date).date()
        filtered_df = df[
        (df['daily_date'].dt.date >= start_date) & 
        (df['daily_date'].dt.date <= end_date) &
        (df['age_range'] >= start_age_range) & 
        (df['age_range'] <= end_age_range) &        
        (df['platform'] >= start_platform) &
        (df['platform'] <= end_platform)
        ]
        return filtered_df

    # Filter out 'Unknown' values from the age_range column
    filtered_age_ranges = df[df['age_range'] != 'Unknown']['age_range']

    # Sort the unique age range values with a custom sorting function
    unique_age_ranges = sorted(filtered_age_ranges.unique(), key=lambda x: [int(age) for age in x.split('-')])
    # Append 'Unknown' to the end of the sorted list
    unique_age_ranges.append('Unknown')     

    # Define your default start and end age ranges
    default_start_age_range = '18-24'
    default_end_age_range = 'Unknown'
    
    start_age_range_picker = widgets.Dropdown(
        description='Start Age Range:',
        options=unique_age_ranges,
        value=default_start_age_range,
        disabled=False
    )
    end_age_range_picker = widgets.Dropdown(
        description='End Age Range:',
        options=unique_age_ranges,
        value=default_end_age_range,
        disabled=False
    )    
    # Define your default start and end platform ranges
    default_start_platform_range = 'android'
    default_end_platform_range = 'web'
        
    start_platform_range_picker = widgets.Dropdown(
        description='Start Platform:',
        options=df['platform'].unique(),
        value=default_start_platform_range,
        disabled=False
    )
    end_platform_range_picker = widgets.Dropdown(
        description='End Platform:',
        options=df['platform'].unique(),
        value=default_end_platform_range,
        disabled=False
    )

    # Update function for the line plot
    def update_line_plot(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform, total):
        if total:
            # If the "Merge Platforms" checkbox is checked, calculate the total rides and cancellations
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)
            total_reviews = filtered_data.groupby('daily_date')['total_reviews'].sum().reset_index()
            avg_rating = filtered_data.groupby('daily_date')['avg_rating'].mean().reset_index()
        
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=total_reviews['daily_date'], y=total_reviews['total_reviews'], name='Reviews'))
            fig.add_trace(go.Scatter(x=avg_rating['daily_date'], y=avg_rating['avg_rating'], name='Rating'))
        else:
            # If "Merge Platforms" is unchecked, display the data per platform
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)

            fig = go.Figure()

            platforms = filtered_data['platform'].unique()
        
            for platform in platforms:
                platform_data = filtered_data[filtered_data['platform'] == platform]
                platform_data_reviews = platform_data.groupby('daily_date')['total_reviews'].sum().reset_index()
                platform_data_ratings = platform_data.groupby('daily_date')['avg_rating'].mean().reset_index()
                fig.add_trace(go.Scatter(x=platform_data_reviews['daily_date'], y=platform_data_reviews['total_reviews'], name=f'{platform} Reviews'))
                fig.add_trace(go.Scatter(x=platform_data_ratings['daily_date'], y=platform_data_ratings['avg_rating'], name=f'{platform} Rating'))
    
        fig.update_layout(title="Number of Rides and USD per Platform",
                          xaxis_title="Date",
                          yaxis_title="Counts")
    
        fig.show()

    # Create an interactive widget to update the line plot
    widget = interactive(
        update_line_plot, 
        start_date=start_date_picker, 
        end_date=end_date_picker, 
        start_age_range=start_age_range_picker, 
        end_age_range=end_age_range_picker,        
        start_platform=start_platform_range_picker, 
        end_platform=end_platform_range_picker,
        total=total_checkbox
    )

    # Display the widgets
    display(widget)
    


In [77]:
def display_charge_users_drivers():
    # Load the dataset from the CSV file
    df = pd.read_csv('D:\\MetroCar\\CSV\\chargestatistics.csv')

    # Create a grouped bar chart
    fig = go.Figure()

    # Define the unique charge_status values
    charge_statuses = df['charge_status'].unique()

    # Loop through each charge_status
    for charge_status in charge_statuses:
        charge_status_data = df[df['charge_status'] == charge_status]
        users_count = charge_status_data['users'].values[0]
        drivers_count = charge_status_data['drivers'].values[0]
        fig.add_trace(go.Bar(
            x=['Users', 'Drivers'],
            y=[users_count, drivers_count],
            name=charge_status,
            text=[users_count, drivers_count],
            textposition='auto'
        ))

    # Update the layout
    fig.update_layout(
        title='Charge Status Comparison for Users and Drivers',
        xaxis_title='Charge Status',
        yaxis_title='Count',
        barmode='group'
    )

    # Show the chart
    fig.show()


In [78]:
def display_charge_rides_reviews():
    # Load the dataset from the CSV file
    df = pd.read_csv('D:\\MetroCar\\CSV\\chargestatistics.csv')

    # Create a grouped bar chart
    fig = go.Figure()

    # Define the unique charge_status values
    charge_statuses = df['charge_status'].unique()

    # Loop through each charge_status
    for charge_status in charge_statuses:
        charge_status_data = df[df['charge_status'] == charge_status]
        rides_count = charge_status_data['rides'].values[0]
        reviews_count = charge_status_data['reviews'].values[0]
        fig.add_trace(go.Bar(
            x=['Rides', 'Reviews'],
            y=[rides_count, reviews_count],
            name=charge_status,
            text=[rides_count, reviews_count],
            textposition='auto'
        ))

    # Update the layout
    fig.update_layout(
        title='Charge Status Comparison for Rides and Reviews',
        xaxis_title='Charge Status',
        yaxis_title='Count',
        barmode='group'
    )

    # Show the chart
    fig.show()


In [79]:
def display_charge_totalusd():
    # Load the dataset from the CSV file
    df = pd.read_csv('D:\\MetroCar\\CSV\\chargestatistics.csv')

    # Create a grouped bar chart
    fig = go.Figure()

    # Define the unique charge_status values
    charge_statuses = df['charge_status'].unique()

    # Loop through each charge_status
    for charge_status in charge_statuses:
        charge_status_data = df[df['charge_status'] == charge_status]
        totalusd = charge_status_data['total_usd'].values[0]
        fig.add_trace(go.Bar(
            x=[charge_status],
            y=[totalusd],
            name=charge_status,
            text=[totalusd],
            textposition='auto'
        ))

    # Update the layout
    fig.update_layout(
        title='Total USD comparison for Charge Status',
        xaxis_title='Charge Status',
        yaxis_title='Total USD',
        barmode='group'
    )

    # Show the chart
    fig.show()


In [80]:
def display_usd_users():
    # Load the dataset from the CSV file
    df = pd.read_csv('D:\\MetroCar\\CSV\\user_declines.csv')

    # Create a slider widget for filtering lost_total_usd values
    lost_total_usd_slider = widgets.FloatSlider(
        value=0,  # Set the initial value to 0
        min=0,
        max=df['lost_total_usd'].max(),
        step=1,
        description='Lost Total USD Filter:',
        style={'description_width': 'initial'},
        layout=Layout(width='50%')
    )
    gained_total_usd_slider = widgets.FloatSlider(
        value=0,  # Set the initial value to 0
        min=0,
        max=df['gained_total_usd'].max(),
        step=1,
        description='Gained Total USD Filter:',
        style={'description_width': 'initial'},
        layout=Layout(width='50%')
    )

    # Create an output widget to display the chart
    output = Output()

    # Function to create the scatter plot using Plotly Graph Objects
    def create_usd_users_scatter(lost_total_usd_filter, gained_total_usd_filter):
        # Filter the data based on the slider value
        filtered_data = df[(df['lost_total_usd'] >= lost_total_usd_filter) & (df['gained_total_usd'] >= gained_total_usd_filter)]

        # Create a scatter plot using Plotly Graph Objects
        fig = go.Figure()

        # Add scatter traces for lost_total_usd and gained_total_usd
        fig.add_trace(go.Scatter(
            x=filtered_data['lost_total_usd'],
            y=filtered_data['user_id'],
            mode='markers',
            marker=dict(size=10, color='red'),  # Customize marker color
            name='Lost Total USD'
        ))

        fig.add_trace(go.Scatter(
            x=filtered_data['gained_total_usd'],
            y=filtered_data['user_id'],
            mode='markers',
            marker=dict(size=10, color='blue'),  # Customize marker color
            name='Gained Total USD'
        ))

        fig.update_layout(
            title='Lost and Gained Total USD comparison By users',
            xaxis_title='Total USD',
            yaxis_title='User ID'
        )

        return fig

    # Function to update the scatter plot based on the slider value
    def update_usd_users_scatter(lost_total_usd_filter, gained_total_usd_filter):
        # Clear the previous output
        with output:
            output.clear_output()

        # Get the Plotly figure
        fig = create_usd_users_scatter(lost_total_usd_filter, gained_total_usd_filter)

        # Show the scatter plot
        fig.show()

    # Create an interactive widget to update the scatter plot based on the slider value
    widget = interactive(update_usd_users_scatter, lost_total_usd_filter=lost_total_usd_slider, gained_total_usd_filter=gained_total_usd_slider)

    # Display the widgets
    display(widget)

In [81]:
def display_usd_drivers():
    # Load the dataset from the CSV file
    df = pd.read_csv('D:\\MetroCar\\CSV\\driver_declines.csv')

    # Create a slider widget for filtering lost_total_usd values
    lost_total_usd_slider = widgets.FloatSlider(
        value=0,  # Set the initial value to 0
        min=0,
        max=df['lost_total_usd'].max(),
        step=1,
        description='Lost Total USD Filter:',
        style={'description_width': 'initial'},
        layout=Layout(width='50%')
    )
    gained_total_usd_slider = widgets.FloatSlider(
        value=0,  # Set the initial value to 0
        min=0,
        max=df['gained_total_usd'].max(),
        step=1,
        description='Gained Total USD Filter:',
        style={'description_width': 'initial'},
        layout=Layout(width='50%')
    )

    # Create an output widget to display the chart
    output = Output()

    # Function to create the scatter plot using Plotly Graph Objects
    def create_usd_drivers_scatter(lost_total_usd_filter, gained_total_usd_filter):
        # Filter the data based on the slider value
        filtered_data = df[(df['lost_total_usd'] >= lost_total_usd_filter) & (df['gained_total_usd'] >= gained_total_usd_filter)]

        # Create a scatter plot using Plotly Graph Objects
        fig = go.Figure()

        # Add scatter traces for lost_total_usd and gained_total_usd
        fig.add_trace(go.Scatter(
            x=filtered_data['lost_total_usd'],
            y=filtered_data['driver_id'],
            mode='markers',
            marker=dict(size=10, color='red'),  # Customize marker color
            name='Lost Total USD'
        ))

        fig.add_trace(go.Scatter(
            x=filtered_data['gained_total_usd'],
            y=filtered_data['driver_id'],
            mode='markers',
            marker=dict(size=10, color='blue'),  # Customize marker color
            name='Gained Total USD'
        ))

        fig.update_layout(
            title='Lost and Gained Total USD comparison By Drivers',
            xaxis_title='Total USD',
            yaxis_title='Driver ID'
        )

        return fig

    # Function to update the scatter plot based on the slider value
    def update_usd_drivers_scatter(lost_total_usd_filter, gained_total_usd_filter):
        # Clear the previous output
        with output:
            output.clear_output()

        # Get the Plotly figure
        fig = create_usd_drivers_scatter(lost_total_usd_filter, gained_total_usd_filter)

        # Show the scatter plot
        fig.show()

    # Create an interactive widget to update the scatter plot based on the slider value
    widget = interactive(update_usd_drivers_scatter, lost_total_usd_filter=lost_total_usd_slider, gained_total_usd_filter=gained_total_usd_slider)

    # Display the widgets
    display(widget)

In [82]:
# Define the main categories and their respective subcategories
categories = {
    'Funnel': ['Funnel Visualization'],
    'Downloads': ['Daily Downloads by Platform'],
    'Signups': [
                'Signups by Age-Range & Platform',
                'Daily Signups by Platform'
               ],
    'Rides Requested': [
                        'Ride Requests by Age-Range & Platform',
                        'Ride Requests & Cancellations Daily'
                       ],
    'Rides Completed': [
                        'Rides Completed by Age-Range & Platform',
                        'USD by Age-Range & Platform',
                        'Rides Completed & USD Daily',
                        'Average Rating by Age-Range & Platform',
                        'Reviews & Average Rating Daily',
                       ],
    'Charge Status':[
                     'Charge Status By Users & Drivers', # Aggregate users + drivers in a bar chart
                     'Charge Status By Rides & Reviews', # Agreggate rides + reviews in a bar chart
                     'Charge Status By Total USD',
                    ],
    'Accepted & Declined':[
                            'USD Lost & Gained By Users', # bar? with a slide bar to filter usd
                            'USD Lost & Gained By Drivers', # bar? with a slide bar to filter usd
                          ],
    
}

# Create dropdown widgets for main categories and subcategories
main_category_selector = widgets.Dropdown(
    options=categories.keys(),
    description='Main Category:'
)

subcategory_selector = widgets.Dropdown(
    options=categories[main_category_selector.value],
    description='Subcategory:'
)

# Function to update subcategories based on the selected main category
def update_subcategories(change):
    subcategory_selector.options = categories[change.new]

main_category_selector.observe(update_subcategories, names='value')

# Display the selected visualization based on the chosen subcategory
def display_visualization(selected_visualization):
    with output:
        output.clear_output()
        if selected_visualization == 'Funnel Visualization':
            # Display the Funnel Visualization
            display_funnel_visualization()
        elif selected_visualization == 'Daily Downloads by Platform':
            display_download_platform()
        elif selected_visualization == 'Signups by Age-Range & Platform':
            display_signups_visualization()
        elif selected_visualization == 'Daily Signups by Platform':
            display_signupsdaily_visualization()
        elif selected_visualization == 'Ride Requests by Age-Range & Platform':
            display_riderequests_visualization()
        elif selected_visualization == 'Ride Requests & Cancellations Daily':
            display_rides_cancellations_visualization()
        elif selected_visualization == 'Rides Completed by Age-Range & Platform':
            display_completed_age_platform()
        elif selected_visualization == 'USD by Age-Range & Platform':
            display_completed_usd()            
        elif selected_visualization == 'Rides Completed & USD Daily':
            display_completed_usd_daily()            
        elif selected_visualization == 'Average Rating by Age-Range & Platform':
            display_completed_ratings()
        elif selected_visualization == 'Reviews & Average Rating Daily':
            display_completed_reviews_daily()
        elif selected_visualization == 'Charge Status By Users & Drivers':
            display_charge_users_drivers()
        elif selected_visualization == 'Charge Status By Rides & Reviews':
            display_charge_rides_reviews()
        elif selected_visualization == 'Charge Status By Total USD':
            display_charge_totalusd()
        elif selected_visualization == 'USD Lost & Gained By Users':
            display_usd_users() 
        elif selected_visualization == 'USD Lost & Gained By Drivers':
            display_usd_drivers()            

# Set up the event handler for the subcategory value change
widgets.interactive(display_visualization, selected_visualization=subcategory_selector)

# Create an output widget for displaying the visualizations
output = widgets.Output()

# Display the main category and subcategory dropdowns, along with the output widget
display(main_category_selector, subcategory_selector, output)


Dropdown(description='Main Category:', options=('Funnel', 'Downloads', 'Signups', 'Rides Requested', 'Rides Co…

Dropdown(description='Subcategory:', options=('Funnel Visualization',), value='Funnel Visualization')

Output()